In [3]:
# Reference Code : https://data-science-hi.tistory.com/188
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch import nn, optim

In [4]:
data_path = '/content/drive/MyDrive/개인 연구/강의 수강/(패스트 캠퍼스) 데이터 분석/Ch3 Logistic Regression/advertising.csv'
data = pd.read_csv(data_path)

In [5]:
data.head()

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Male,Country,Timestamp,Clicked on Ad
0,68.95,NaN,61833.90,256.09,Cloned 5thgeneration orchestration,Wrightburgh,0,Tunisia,3/27/2016 0:53,0
1,80.23,31.0,68441.85,193.77,Monitored national standardization,West Jodi,1,Nauru,4/4/2016 1:39,0
2,69.47,26.0,59785.94,236.50,Organic bottom-line service-desk,Davidton,0,San Marino,3/13/2016 20:35,0
3,74.15,29.0,54806.18,245.89,Triple-buffered reciprocal time-frame,West Terrifurt,1,Italy,1/10/2016 2:31,0
4,68.37,35.0,73889.99,225.58,Robust logistical utilization,South Manuel,0,Iceland,6/3/2016 3:36,0


In [6]:
# Country Drop
data = data.drop(['Country'], axis=1)

# Add New Feture
data['Time_Spent_Band'] = pd.qcut(data['Daily Time Spent on Site'], 4)
data['Age_Band'] = pd.qcut(data['Age'], 5)
data['Area_Income_Band'] = pd.qcut(data['Area Income'], 7)

data['Timestamp'] = pd.to_datetime(data['Timestamp'], format=('%m/%d/%Y %H:%M'))
data['Year'] = data['Timestamp'].dt.year
data['Month'] =data['Timestamp'].dt.month
data['Day'] = data['Timestamp'].dt.day
data['Hour'] = data['Timestamp'].dt.hour
data['Minute'] = data['Timestamp'].dt.minute

data = data.drop('Timestamp', axis=1)


# Categoric -> Numeric
data = pd.get_dummies(data, columns=['Ad Topic Line', 'City'], drop_first=True)
data = pd.get_dummies(data, columns=['Time_Spent_Band', 'Age_Band', 'Area_Income_Band'], drop_first=True)

In [8]:
data.columns

Index(['Daily Time Spent on Site', 'Age', 'Area Income',
       'Daily Internet Usage', 'Male', 'Clicked on Ad', 'Year', 'Month', 'Day',
       'Hour',
       ...
       'Age_Band_(28.0, 32.0]', 'Age_Band_(32.0, 37.0]',
       'Age_Band_(37.0, 45.0]', 'Age_Band_(45.0, 61.0]',
       'Area_Income_Band_(39806.844, 48887.127]',
       'Area_Income_Band_(48887.127, 54442.166]',
       'Area_Income_Band_(54442.166, 59669.551]',
       'Area_Income_Band_(59669.551, 63994.789]',
       'Area_Income_Band_(63994.789, 68547.109]',
       'Area_Income_Band_(68547.109, 79484.8]'],
      dtype='object', length=1991)

In [98]:
from sklearn.model_selection import train_test_split
X = data.drop('Clicked on Ad', axis=1)
y = data['Clicked on Ad']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

In [53]:
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()
minmax.fit(X_train)

X_train = minmax.transform(X_train).astype(np.float)
X_test = minmax.transform(X_test).astype(np.float)

print(X_train.shape)
print(X_test.shape)

(800, 1990)
(200, 1990)


In [78]:
X_train

array([[0.84956655, 0.45238095, 0.78640429, ..., 0.        , 1.        ,
        0.        ],
       [0.31922489, 0.95238095, 0.43314592, ..., 0.        , 0.        ,
        0.        ],
       [0.73006969, 0.4047619 , 0.90456417, ..., 0.        , 0.        ,
        1.        ],
       ...,
       [0.51861295, 0.35714286, 0.74992602, ..., 1.        , 0.        ,
        0.        ],
       [0.4072752 , 0.16666667, 0.83659142, ..., 0.        , 0.        ,
        1.        ],
       [0.2381438 , 0.54761905, 0.79190084, ..., 0.        , 1.        ,
        0.        ]])

In [54]:
y_train.to_numpy().shape

(800,)

In [99]:
X_train = X_train.to_numpy().astype(np.float)
X_test = X_test.to_numpy().astype(np.float)
y_train = y_train.to_numpy().astype(np.float).reshape(-1,1)
y_test = y_test.to_numpy().astype(np.float).reshape(-1,1)

In [56]:
input_dim = X_train.shape[1]
hidden_dim = int(input_dim/4)
output_dim = 1

In [114]:
import torch.nn.functional as F
sigmoid = nn.Sigmoid()
relu = nn.ReLU()
bn1 = nn.BatchNorm1d(hidden_dim)
model = nn.Sequential(nn.Linear(input_dim, output_dim), sigmoid)#, \
                      #bn1,
                      #relu)# \
                      #nn.Linear(hidden_dim, output_dim), \
                      #sigmoid)

optimizer = optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.BCELoss()

In [100]:
from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
test_dataset = TensorDataset(torch.tensor(X_test), torch.tensor(y_test))

In [101]:
train_loader = DataLoader(train_dataset, batch_size = 10, shuffle=False    )
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

In [115]:
list_loss = []
for epoch in range(100):
  batch_loss = 0.0
  for xx, yy in train_loader:
    optimizer.zero_grad()
    print(xx.float()[4])
    y_pred = model(xx.float())
    print(y_pred)
    #y_pred = torch.nan_to_num(y_pred)
    loss = loss_fn(y_pred, yy.float())
    loss.backward()   # backprop 계산
    optimizer.step()  # Optim 계산
    batch_loss += loss.item()
  list_loss.append(batch_loss)

tensor([8.7850e+01, 3.4000e+01, 5.1816e+04,  ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00])
tensor([[0.],
        [0.],
        [0.],
        [nan],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], grad_fn=<SigmoidBackward>)


RuntimeError: ignored

In [116]:
for xx, yy in test_loader:
  with torch.no_grad():
    prob = model(xx.float())
    predict = (prob>0.5).float()
    acc = (predict==yy).float().mean()
    print(acc)

tensor(0.4000)
tensor(0.6000)
tensor(0.5000)
tensor(0.3000)
tensor(0.8000)
tensor(0.6000)
tensor(0.4000)
tensor(0.6000)
tensor(0.4000)
tensor(0.5000)
tensor(0.5000)
tensor(0.5000)
tensor(0.3000)
tensor(0.4000)
tensor(0.7000)
tensor(0.5000)
tensor(0.6000)
tensor(0.5000)
tensor(0.6000)
tensor(0.3000)
